This is preprocessing with custom functions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# for Tina machine only, to import .py from src/ds/
import os
current_workpath = os.getcwd()
print("Current workpath:", current_workpath)

parent_folder = os.path.dirname(current_workpath)
print("Parent_folder:", parent_folder)

import sys
sys.path.append(parent_folder+'/src')
print(sys.path[-1])

Current workpath: /home/tina4aiml/dev/notebooks
Parent_folder: /home/tina4aiml/dev
/home/tina4aiml/dev/src


In [3]:
rawfile_name = 'beer_reviews'
rawfile_format = '.csv'
rawfile_path = "../data/raw/"
datadict_path = "../references/Data_Dict.csv"

interim_folder_path = "../data/interim/"
processed_folder_path  ="../data/processed/"
model_folder_path = "../models/"

In [4]:
import pandas as pd
import numpy as np

In [5]:
df_raw = pd.read_csv(rawfile_path + rawfile_name + rawfile_format)
df_raw.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [6]:
data_dict = pd.read_csv(datadict_path)
features_drop = data_dict.loc[data_dict['API Expected Parameter'] == 'No', 'Column'].tolist()
features_drop.remove('beer_style (target)')

In [7]:
df_prep = df_raw.drop(features_drop, axis=1)

In [8]:
from ds.data.sets import pop_target

In [9]:
df, target = pop_target(df_prep, 'beer_style', to_numpy=False)
np.save(processed_folder_path + 'beer_style', target.unique())

In [10]:
df.head()

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv
0,Vecchio Birraio,2.0,2.5,1.5,1.5,5.0
1,Vecchio Birraio,2.5,3.0,3.0,3.0,6.2
2,Vecchio Birraio,2.5,3.0,3.0,3.0,6.5
3,Vecchio Birraio,3.0,3.5,2.5,3.0,5.0
4,Caldera Brewing Company,4.5,4.0,4.0,4.5,7.7


In [11]:
target.head()

0                        Hefeweizen
1                English Strong Ale
2            Foreign / Export Stout
3                   German Pilsener
4    American Double / Imperial IPA
Name: beer_style, dtype: object

In [12]:
df_features = df.columns.tolist()
np.save(processed_folder_path + 'features_label', df_features)
np.save(processed_folder_path + 'brewery_name', df['brewery_name'].unique())

# Preprocess x_df

In [13]:
from ds.data.sets import DataProcessor

In [14]:
from sklearn.preprocessing import StandardScaler
#from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer


In [15]:
imputer = SimpleImputer(strategy="mean")
scaler = StandardScaler()
# knn_imputer_numeric = KNNImputer(n_neighbors = 10)  

In [16]:
#scaler and knn_imputer_numeric will save under '../models/'
data_processor = DataProcessor(scaler, imputer)

In [17]:
X_proceesed = data_processor.process_dataframe(df,  dest = "../data/interim/", hashbuckets = 1000)

hashing
imputing
scaling
concat
array
saving


In [18]:
# contains 12 columns, 6 orginal columns and 6 pre-process columns
df_processed = pd.DataFrame(X_proceesed[1])

In [19]:
df_processed 

,0,1,2,3,4,5
0,286.0,-2.487952,-2.177663,-3.288833,-3.132454,-0.898791
1,286.0,-1.771225,-1.366096,-1.090123,-1.083188,-0.370708
2,286.0,-1.771225,-1.366096,-1.090123,-1.083188,-0.238688
3,286.0,-1.054499,-0.554530,-1.823026,-1.083188,-0.898791
4,95.0,1.095679,0.257037,0.375684,0.966078,0.289395
...,...,...,...,...,...,...
1586609,752.0,0.378953,-0.554530,0.375684,0.282989,-0.810777
1586610,752.0,1.812405,-2.177663,-2.555929,0.282989,-0.810777
1586611,752.0,-0.337773,-1.366096,-0.357219,0.282989,-0.810777
1586612,752.0,1.095679,1.068603,1.108588,0.966078,-0.810777


In [20]:
df_features
df_processed.columns = df_features

In [21]:
df_processed

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv
0,286.0,-2.487952,-2.177663,-3.288833,-3.132454,-0.898791
1,286.0,-1.771225,-1.366096,-1.090123,-1.083188,-0.370708
2,286.0,-1.771225,-1.366096,-1.090123,-1.083188,-0.238688
3,286.0,-1.054499,-0.554530,-1.823026,-1.083188,-0.898791
4,95.0,1.095679,0.257037,0.375684,0.966078,0.289395
...,...,...,...,...,...,...
1586609,752.0,0.378953,-0.554530,0.375684,0.282989,-0.810777
1586610,752.0,1.812405,-2.177663,-2.555929,0.282989,-0.810777
1586611,752.0,-0.337773,-1.366096,-0.357219,0.282989,-0.810777
1586612,752.0,1.095679,1.068603,1.108588,0.966078,-0.810777


In [22]:
df_processed.to_pickle(interim_folder_path + 'df_processed.pkl')

# Split processed into train, test and val, save as np array

In [23]:
df_processed = pd.read_pickle(interim_folder_path + 'df_processed.pkl')

In [24]:
from ds.data.sets import split_sets_random

In [25]:
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(df_processed, target=target, test_ratio=0.2, to_numpy=True)

In [26]:
X_train[:5]

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv
1388520,810.0,-3.204678,-1.366096,-1.090123,-0.400099,-1.118826
574735,830.0,-1.054499,0.257037,-0.357219,-0.400099,0.641450
235720,397.0,-1.054499,-0.554530,0.375684,-0.400099,0.069361
1087888,55.0,-0.337773,0.257037,-1.823026,-0.400099,1.213540
282858,193.0,0.378953,-0.554530,-0.357219,-0.400099,-0.942798


In [27]:
# Use index number from the post transform dataset to get the original record from the raw dataset
X_train_v3_unencoded = df.loc[X_train.index]

print(X_train_v3_unencoded)

                               brewery_name  review_aroma  review_appearance  \
1388520      Kirin Brewery Company, Limited           1.5                3.0   
574735          Huisbrouwerij Klein Duimpje           3.0                4.0   
235720            Southampton Publick House           3.0                3.5   
1087888    Rock Bottom Restaurant & Brewery           3.5                4.0   
282858   Boston Beer Company (Samuel Adams)           4.0                3.5   
...                                     ...           ...                ...   
912020   Bierbrouwerij Sint Christoffel B.V           4.5                4.0   
1083305              Brouwerij Slaghmuylder           3.5                4.0   
460341          Thirsty Dog Brewing Company           4.0                4.0   
626118         OPA-OPA Steakhouse & Brewery           4.0                3.5   
858708                        Mikkeller ApS           4.0                4.5   

         review_palate  review_taste  b

In [28]:
X_train_v3_unencoded.to_pickle(interim_folder_path + 'df_X_train_v3.pkl')

In [29]:
print(f"Total of {len(X_train)} records, with {X_train['brewery_name'].nunique()} unqiue brewery_names in X_train, and {y_train.nunique()} unique beer_style in y_train.")

Total of 951968 records, with 993 unqiue brewery_names in X_train, and 104 unique beer_style in y_train.


In [30]:
X_val[:5]

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv
560133,240.0,-1.054499,-1.366096,0.375684,-1.083188,-1.074819
1414694,123.0,-1.054499,0.257037,-1.090123,-1.083188,0.861485
553164,664.0,0.378953,-0.554530,0.375684,0.282989,-0.810777
1471382,75.0,-0.337773,1.068603,-0.357219,0.282989,1.301554
1210217,138.0,-0.337773,1.880169,1.108588,0.282989,0.201381


In [31]:
# Use index number to subset df2 based on df1's index
X_val_v3_unencoded = df.loc[X_val.index]

print(X_val_v3_unencoded)

                          brewery_name  review_aroma  review_appearance  \
560133           Boulevard Brewing Co.           3.0                3.0   
1414694           Shipyard Brewing Co.           3.0                4.0   
553164        Nebraska Brewing Company           4.0                3.5   
1471382        Les Trois Mousquetaires           3.5                4.5   
1210217                Heavy Seas Beer           3.5                5.0   
...                                ...           ...                ...   
194715           Eel River Brewing Co.           2.5                3.5   
1556120        Victory Brewing Company           4.5                4.0   
921472            Vale Brewery Company           3.5                4.0   
1134381  Russian River Brewing Company           4.5                5.0   
412572     Great Lakes Brewing Company           4.0                4.5   

         review_palate  review_taste  beer_abv  
560133             4.0           3.0       4.6  
1

In [32]:
X_val_v3_unencoded.to_pickle(interim_folder_path + 'df_X_val_v3.pkl')

In [33]:
print(f"Total of {len(X_val)} records, with {X_val['brewery_name'].nunique()} unqiue brewery_names in X_val, and {y_val.nunique()} unique beer_style in y_val.")

Total of 317323 records, with 985 unqiue brewery_names in X_val, and 104 unique beer_style in y_val.


In [34]:
X_test[:5]

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv
372797,745.0,0.378953,1.880169,0.375684,0.966078,0.861485
942896,841.0,0.378953,1.068603,1.841491,0.282989,1.521588
1152395,836.0,-0.337773,-0.554530,-1.090123,0.966078,-0.546736
410631,275.0,1.095679,0.257037,1.108588,0.966078,0.201381
699913,218.0,-0.337773,0.257037,-1.090123,-1.766277,-0.485126


In [35]:
# Use index number to subset df2 based on df1's index
X_test_v3_unencoded = df.loc[X_test.index]

print(X_test_v3_unencoded)

                                    brewery_name  review_aroma  \
372797                                  Unibroue           4.0   
942896                      Bell's Brewery, Inc.           4.0   
1152395                  Arcadia Brewing Company           3.5   
410631                    Tröegs Brewing Company           4.5   
699913             Boulder Beer / Wilderness Pub           3.5   
...                                          ...           ...   
1126739                       Miller Brewing Co.           2.0   
101744                 Hale's Ales Brewery & Pub           4.0   
690490   Bayerische Staatsbrauerei Weihenstephan           4.0   
554111                    Summit Brewing Company           3.5   
368888                                  Unibroue           4.0   

         review_appearance  review_palate  review_taste  beer_abv  
372797                 5.0            4.0           4.5      9.00  
942896                 4.5            5.0           4.0     10.50  
115

In [36]:
X_test_v3_unencoded.to_pickle(interim_folder_path + 'df_X_test_v3_unencoded.pkl')

In [37]:
print(f"Total of {len(X_test)} records, with {X_test['brewery_name'].nunique()} unqiue brewery_names in X_test, and {y_test.nunique()} unique beer_style in y_test.")

Total of 317323 records, with 985 unqiue brewery_names in X_test, and 104 unique beer_style in y_test.


In [38]:
print(X_train.shape) #(951968, 6)
print(X_val.shape) #(317323, 6)
print(X_test.shape) #(317323, 6)

(951968, 6)
(317323, 6)
(317323, 6)


In [39]:
from ds.data.sets import save_sets_v2

In [40]:
save_sets_v2(X_train, y_train, X_val, y_val, X_test, y_test, path='../data/processed/', suffix='_v3')

Start from the beer_reviews.csv, by the end of this, we should have 

1) array of unique 'beer_style' saved in "../data/processed/" as "beer_style.npy"
2) array of unqiue 'brewery_name' in "../data/processed/"  as "brewery_name.npy"
3) array of feature names in "../data/processed/" as "features_label.npy"
4) the numeric imputer saved in "../models/" as imputer_numeric.joblib
5) the numberic scaler saved in "../models/" as scaler.joblib
6) array of the fully preprocessed data saved in "../data/interim/" as 'X_processed.npy'
7) df of fully processd df in "../data/interim/" as "df_processed.pkl"
8) Train/val/test datasets split from "df_processed" in "../data/processed/"
